In [1]:
import scipy.sparse as sparse
import scipy.io as sio
import scipy.stats as stats
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
from time import time
import pandas as pd
# own code
import libarary.graph_tools as gtl
import libarary.invarients as inva
import libarary.cpp as cpp

In [2]:
cpp.test()

hello world


In [3]:
def test(G: nx.Graph):
    clocks = []
    start = time()
    dists = gtl.convert_dist_dict_to_narray(
        dict(nx.all_pairs_dijkstra_path_length(G)), list(G.nodes)
    )
    end = time()
    clocks.append(end - start)
    print("clock convert: ", clocks[-1])
    start = time()
    q_extend3 = cpp.q_extend_single(dists, 3)
    end = time()
    clocks.append(end - start)
    print("clock q_extend3: ", clocks[-1], q_extend3)
    start = time()
    excess = cpp.excess_global_single(dists)
    end = time()
    clocks.append(end - start)
    print("clock excess_global: ", clocks[-1], excess)
    start = time()
    q_packing3 = cpp.q_packing_single(dists, 3)
    end = time()
    clocks.append(end - start)
    print("clock q_packing: ", clocks[-1], q_packing3)

    res = {
        "|V|": G.number_of_nodes(),
        "q-extend3": q_extend3,
        "excess-global": excess,
        "q-packing3": q_packing3,
    }
    return res


res = {}


# mouse brain

In [4]:
G = gtl.read_graph_from_edgelist("networks-random/brain274/bn-mouse_brain_1.edges")
print("number of nodes: ", G.number_of_nodes())

res["mouse-brain"] = test(G)


number of nodes:  213
clock convert:  0.5452790260314941
clock q_extend3:  0.5374517440795898 0.0001771636105943839
clock excess_global:  0.30487680435180664 3.0
clock q_packing:  0.5534834861755371 1.0


{'mouse-brain': {'|V|': 213,
  'q-extend3': 0.0001771636105943839,
  'excess-global': 3.0,
  'q-packing3': 1.0}}

# aves social

In [5]:
g1 = gtl.read_graph_from_edgelist("networks-random/aves-wildbirds/aves-wildbird-network.edges")
print(g1.number_of_nodes())
res["aves"] = test(g1)

202
clock convert:  0.1606743335723877
clock q_extend3:  0.45962047576904297 0.0003940692576720359
clock excess_global:  0.26184558868408203 7.0
clock q_packing:  0.466428279876709 0.5


# Mammalia social connections

In [27]:
g2 = gtl.read_graph_from_edgelist(
        "networks-random/memalia-big/mammalia-voles-bhp-trapping.edges"
    )
print("number of nodes: ", g2.number_of_nodes())
res["Mammalia social"] = test(g2)


number of nodes:  1686
clock convert:  2.543473720550537


# Power Eris

In [7]:
mm = sio.mmread("networks1/power/power-eris1176.mtx")
g3: nx.Graph = nx.from_scipy_sparse_array(mm)

print("number of nodes: ", g3.number_of_nodes())

res["power-eris"] = test(g3)


number of nodes:  1176
clock convert:  4.978182315826416
clock q_extend3:  89.9827823638916 0.00015342307135620206
clock excess_global:  54.07833242416382 105.0
clock q_packing:  92.11993050575256 1.0


# Results

In [9]:
ds = pd.DataFrame(res)
ds

,mouse-brain,aves,power-eris
|V|,213.000000,202.000000,1176.000000
q-extend3,0.000177,0.000394,0.000153
excess-global,3.000000,7.000000,105.000000
q-packing3,1.000000,0.500000,1.000000
